
# Exact matches model

1. **For each training set sequence,  for each disease class, run Fisher’s exact test on 2x2 contingency table** counting the number of unique participants that are/aren't in a disease class and have/don't have a particular V/J/CDR3.


```
Contigency table is:
---------------------------------------------|---------------------- |
is other class |                                                     |
----------------      # unique participants with this property       -
is this class  |                                                     |
---------------|-----------------------------|---------------------- |
               | does not have this sequence | has this sequence --- |
```

This returns p-values representing how enriched each sequence is in each disease class.

No multiple hypothesis correction, but we do try some low p-value thresholds below that may simulate Bonferroni correction.

2. **Given a p-value threshold, featurize and fit a model.**

Featurization into `n_specimens x n_diseases` feature matrix:
- For a specimen, count how many of its sequences are exact matches to [sequences whose Covid-19 specificity p-values <= p-value threshold], [sequences whose HIV specificity p-values <= p-value threshold], etc.
- Divide by total number of sequences in the specimen




3. **Evaluate on validation set. Choose p-value threshold that gives highest AUC.**


4. **Evaluate best model on test set, possibly after tuning decision thresholds on validation set.**

Note that we explicitly find Healthy-specific sequences in the featurization.

# Analyze exact matches model performance on train_smaller2, the held out set used to tune the model's hyperparameters

In [1]:
from IPython.display import display, Markdown
from malid import config, logger, io
import crosseval
from malid.datamodels import (
    map_cross_validation_split_strategy_to_default_target_obs_column,
)
from malid.trained_model_wrappers import (
    ExactMatchesClassifier,
    ConvergentClusterClassifier,
)
import joblib
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import matplotlib.ticker
from summarynb import show, indexed_csv, table, chunks

## Analyze on train_smaller2, the held out set used to tune the model's hyperparameters

In [2]:
# Run this benchmark model on the default classification target only, e.g. TargetObsColumnEnum.disease
target_obs_column = map_cross_validation_split_strategy_to_default_target_obs_column[
    config.cross_validation_split_strategy
]

In [3]:
for gene_locus in config.gene_loci_used:
    target_obs_column.confirm_compatibility_with_gene_locus(gene_locus)
    target_obs_column.confirm_compatibility_with_cross_validation_split_strategy(
        config.cross_validation_split_strategy
    )
    models_base_dir = ExactMatchesClassifier._get_model_base_dir(
        gene_locus=gene_locus, target_obs_column=target_obs_column
    )  # should already exist

    output_base_dir = ExactMatchesClassifier._get_output_base_dir(
        gene_locus=gene_locus,
        target_obs_column=target_obs_column,
    )  # might not yet exist
    output_base_dir.mkdir(parents=True, exist_ok=True)  # create if needed

    model_output_prefix = models_base_dir / "train_smaller1_model"
    results_output_prefix = output_base_dir / "train_smaller1_model"

    try:
        logger.info(
            f"{gene_locus}, {target_obs_column} from {model_output_prefix} to {results_output_prefix}"
        )

        ## Load and summarize
        experiment_set = crosseval.ExperimentSet.load_from_disk(
            output_prefix=model_output_prefix
        )

        # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
        # TODO: make kdict support: del self.model_outputs[:, fold_id]
        for key in experiment_set.model_outputs[:, -1].keys():
            logger.debug(f"Removing {key} (global fold)")
            del experiment_set.model_outputs[key]

        experiment_set_global_performance = experiment_set.summarize()
        experiment_set_global_performance.export_all_models(
            func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.txt",
            func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.png",
            dpi=72,
        )
        combined_stats = experiment_set_global_performance.get_model_comparison_stats()
        combined_stats.to_csv(
            f"{results_output_prefix}.compare_model_scores.tsv",
            sep="\t",
        )
        print(gene_locus, target_obs_column)
        display(combined_stats)

    except Exception as err:
        logger.exception(f"{gene_locus}, {target_obs_column} failed with error: {err}")

{"message": "GeneLocus.BCR, TargetObsColumnEnum.disease from /home/maxim/code/immune-repertoire-classification/data/data_v_20231027/in_house_peak_disease_timepoints/exact_matches/BCR/disease/train_smaller1_model to /home/maxim/code/immune-repertoire-classification/out/in_house_peak_disease_timepoints/exact_matches/BCR/disease/train_smaller1_model", "time": "2023-11-04T02:43:27.462408"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:40.057727"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:40.085678"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:40.110327"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:40.122870"}


...

{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:52.703278"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:52.727847"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T02:43:52.740150"}


GeneLocus.TCR TargetObsColumnEnum.disease


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
ridge_sklearn_with_lambdamax,0.775 +/- 0.048 (in 3 folds),0.771 +/- 0.054 (in 3 folds),0.811 +/- 0.044 (in 3 folds),0.809 +/- 0.048 (in 3 folds),0.417 +/- 0.039 (in 3 folds),0.250 +/- 0.037 (in 3 folds),0.417,0.221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
ridge_cv,0.775 +/- 0.048 (in 3 folds),0.770 +/- 0.053 (in 3 folds),0.811 +/- 0.044 (in 3 folds),0.809 +/- 0.048 (in 3 folds),0.417 +/- 0.039 (in 3 folds),0.250 +/- 0.037 (in 3 folds),0.417,0.221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
ridge_cv_lambda1se,0.774 +/- 0.049 (in 3 folds),0.770 +/- 0.054 (in 3 folds),0.811 +/- 0.044 (in 3 folds),0.808 +/- 0.048 (in 3 folds),0.417 +/- 0.039 (in 3 folds),0.251 +/- 0.036 (in 3 folds),0.417,0.222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet0.25_sklearn_with_lambdamax,0.765 +/- 0.028 (in 3 folds),0.757 +/- 0.035 (in 3 folds),0.807 +/- 0.027 (in 3 folds),0.802 +/- 0.031 (in 3 folds),0.430 +/- 0.045 (in 3 folds),0.286 +/- 0.020 (in 3 folds),0.429,0.267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet_sklearn_with_lambdamax,0.762 +/- 0.025 (in 3 folds),0.753 +/- 0.030 (in 3 folds),0.803 +/- 0.024 (in 3 folds),0.798 +/- 0.025 (in 3 folds),0.434 +/- 0.056 (in 3 folds),0.294 +/- 0.034 (in 3 folds),0.433,0.274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet_cv0.25,0.762 +/- 0.023 (in 3 folds),0.753 +/- 0.029 (in 3 folds),0.803 +/- 0.022 (in 3 folds),0.798 +/- 0.024 (in 3 folds),0.434 +/- 0.038 (in 3 folds),0.288 +/- 0.017 (in 3 folds),0.433,0.271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet_cv0.25_lambda1se,0.761 +/- 0.026 (in 3 folds),0.753 +/- 0.032 (in 3 folds),0.804 +/- 0.023 (in 3 folds),0.799 +/- 0.026 (in 3 folds),0.417 +/- 0.045 (in 3 folds),0.270 +/- 0.026 (in 3 folds),0.417,0.251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet0.75_sklearn_with_lambdamax,0.760 +/- 0.024 (in 3 folds),0.751 +/- 0.029 (in 3 folds),0.800 +/- 0.021 (in 3 folds),0.795 +/- 0.022 (in 3 folds),0.430 +/- 0.053 (in 3 folds),0.290 +/- 0.024 (in 3 folds),0.429,0.270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
logisticregression_unregularized,0.759 +/- 0.025 (in 3 folds),0.749 +/- 0.030 (in 3 folds),0.796 +/- 0.022 (in 3 folds),0.791 +/- 0.024 (in 3 folds),0.410 +/- 0.056 (in 3 folds),0.265 +/- 0.029 (in 3 folds),0.409,0.243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False
elasticnet_cv,0.753 +/- 0.014 (in 3 folds),0.744 +/- 0.018 (in 3 folds),0.794 +/- 0.011 (in 3 folds),0.788 +/- 0.013 (in 3 folds),0.434 +/- 0.046 (in 3 folds),0.286 +/- 0.036 (in 3 folds),0.433,0.269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,0,247,0.000000,False


In [4]:
model_names_to_check_later = combined_stats.index[
    ~combined_stats.index.str.startswith("dummy_")
].tolist()
model_names_to_check_later

['ridge_sklearn_with_lambdamax',
 'ridge_cv',
 'ridge_cv_lambda1se',
 'elasticnet0.25_sklearn_with_lambdamax',
 'elasticnet_sklearn_with_lambdamax',
 'elasticnet_cv0.25',
 'elasticnet_cv0.25_lambda1se',
 'elasticnet0.75_sklearn_with_lambdamax',
 'logisticregression_unregularized',
 'elasticnet_cv',
 'elasticnet_cv_lambda1se',
 'lasso_sklearn_with_lambdamax',
 'linearsvm_ovr',
 'elasticnet_cv0.75_lambda1se',
 'elasticnet_cv0.75',
 'lasso_cv_lambda1se',
 'lasso_cv',
 'elasticnet_cv_each_disease_versus_healthy',
 'xgboost',
 'rf_multiclass',
 'elasticnet_each_disease_versus_healthy_sklearn_with_lambdamax']

# TODO: Evaluate on validation set

# Evaluate on test set

In [5]:
featurized_all = {}
for gene_locus in config.gene_loci_used:
    print(gene_locus)
    results_output_dir = ExactMatchesClassifier._get_output_base_dir(
        gene_locus=gene_locus,
        target_obs_column=target_obs_column,
    )  # already created above when analyzing validation set

    results = crosseval.ExperimentSet()
    for fold_id in config.cross_validation_fold_ids:
        adata_test = io.load_fold_embeddings(
            fold_id=fold_id,
            fold_label="test",
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
            # This model does not require the embedding .X, so take the fast path and just load .obs:
            load_obs_only=True,
        )

        # load validation set for tuning
        adata_validation = io.load_fold_embeddings(
            fold_id=fold_id,
            fold_label="validation",
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
            # This model does not require the embedding .X, so take the fast path and just load .obs:
            load_obs_only=True,
        )

        for model_name in model_names_to_check_later:
            clf = ExactMatchesClassifier(
                fold_id=fold_id,
                model_name=model_name,
                fold_label_train="train_smaller1",
                target_obs_column=target_obs_column,
                gene_locus=gene_locus,
            )

            # Featurization may vary by model, because p-value threshold may vary by model
            featurized: crosseval.FeaturizedData = clf.featurize(adata_test)
            # store for later analysis of featurized.extras
            featurized_all[(gene_locus, fold_id, model_name)] = featurized

            def _make_result(
                name: str,
                model: crosseval.Classifier,
                featurized_test: crosseval.FeaturizedData,
            ):
                # For curiosity's sake, what does the model predict for an input row of all 0s?
                row_of_0s: pd.DataFrame = (
                    pd.Series(0, index=model.feature_names_in_).to_frame().T
                )
                row_of_0s_predict = model.predict(row_of_0s)[0]
                row_of_0s_predict_proba = (
                    pd.DataFrame(model.predict_proba(row_of_0s), columns=model.classes_)
                    .iloc[0]
                    .apply(lambda p: f"{p:0.2%}")
                    .to_dict()
                )
                logger.info(
                    f"For a row of all zeros ({row_of_0s.shape}), model {name} on fold {fold_id}, {gene_locus}, {target_obs_column} predicts: {row_of_0s_predict} with probabilities {row_of_0s_predict_proba}"
                )

                # Create performance object
                return crosseval.ModelSingleFoldPerformance(
                    model_name=name,
                    fold_id=fold_id,
                    y_true=featurized_test.y,
                    clf=model,
                    X_test=featurized_test.X,
                    fold_label_train="train_smaller1",
                    fold_label_test="test",
                    test_metadata=featurized_test.metadata,
                    test_abstentions=featurized_test.abstained_sample_y,
                    test_abstention_metadata=featurized_test.abstained_sample_metadata,
                )

            results.add(_make_result(model_name, clf, featurized))

            try:
                # tune on validation set
                clf_tuned = clf.tune_model_decision_thresholds_to_validation_set(
                    validation_set=adata_validation
                )

                # save out tuned model
                joblib.dump(
                    clf_tuned,
                    clf_tuned.models_base_dir
                    / f"train_smaller1_model.{model_name}.decision_thresholds_tuned.{fold_id}.joblib",
                )

                # add tuned model performance
                results.add(
                    _make_result(
                        f"{model_name}_decision_thresholds_tuned", clf_tuned, featurized
                    )
                )
            except Exception as err:
                logger.warning(
                    f"Failed to tune {model_name} on validation set for fold {fold_id}, {gene_locus}, {target_obs_column}"
                )

        # Clear RAM
        del adata_test, adata_validation
        io.clear_cached_fold_embeddings()
        gc.collect()

    # summarize performance across folds and models
    summary = results.summarize()
    combined_stats = summary.get_model_comparison_stats()
    display(combined_stats)

    fname = (
        results_output_dir
        / f"train_smaller1_model.compare_model_scores.test_set_performance.tsv"
    )
    combined_stats.to_csv(fname, sep="\t")

    summary.export_all_models(
        func_generate_classification_report_fname=lambda model_name: results_output_dir
        / f"train_smaller1_model.test_set_performance.{model_name}.classification_report.txt",
        func_generate_confusion_matrix_fname=lambda model_name: results_output_dir
        / f"train_smaller1_model.test_set_performance.{model_name}.confusion_matrix.png",
        dpi=72,
    )

GeneLocus.BCR


{"message": "For a row of all zeros ((1, 6)), model ridge_sklearn_with_lambdamax on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: Healthy/Background with probabilities {'Covid19': '13.65%', 'HIV': '10.60%', 'Healthy/Background': '27.94%', 'Influenza': '3.93%', 'Lupus': '24.48%', 'T1D': '19.40%'}", "time": "2023-11-04T02:46:30.398327"}


{"message": "For a row of all zeros ((1, 6)), model ridge_sklearn_with_lambdamax_decision_thresholds_tuned on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: Healthy/Background with probabilities {'Covid19': '7.84%', 'HIV': '7.69%', 'Healthy/Background': '11.28%', 'Influenza': '2.46%', 'Lupus': '11.21%', 'T1D': '3.80%'}", "time": "2023-11-04T02:46:59.137088"}


{"message": "For a row of all zeros ((1, 6)), model ridge_cv on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: Healthy/Background with probabilities {'Covid19': '13.65%', 'HIV': '10.60%', 'Healthy/Background': '27.95%', 'Influenza': '3.91%', 'Lupus': '24.48%', 'T1D': '19.40%'}", "time": "2023-11-04T02:47:33.711020"}


{"message": "For a row of all zeros ((1, 6)), model ridge_cv_decision_thresholds_tuned on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: Healthy/Background with probabilities {'Covid19': '9.56%', 'HIV': '10.30%', 'Healthy/Background': '15.12%', 'Influenza': '2.70%', 'Lupus': '15.02%', 'T1D': '4.53%'}", "time": "2023-11-04T02:47:58.662556"}


...

{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T06:47:31.865185"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T06:47:31.890943"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T06:47:31.916240"}


{"message": "Inserting phantom class: Healthy/Background", "time": "2023-11-04T06:47:31.930030"}


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
ridge_cv_lambda1se,0.753 +/- 0.019 (in 3 folds),0.745 +/- 0.013 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.421 +/- 0.011 (in 3 folds),0.252 +/- 0.036 (in 3 folds),0.421,0.224,0.420 +/- 0.013 (in 3 folds),0.251 +/- 0.033 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.420,0.223,0.002,Unknown,549,1,550,0.001818,False
ridge_cv_lambda1se_decision_thresholds_tuned,0.753 +/- 0.019 (in 3 folds),0.745 +/- 0.013 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.533 +/- 0.054 (in 3 folds),0.412 +/- 0.046 (in 3 folds),0.534,0.401,0.532 +/- 0.055 (in 3 folds),0.411 +/- 0.048 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.533,0.400,0.002,Unknown,549,1,550,0.001818,False
ridge_sklearn_with_lambdamax,0.753 +/- 0.018 (in 3 folds),0.745 +/- 0.012 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.421 +/- 0.011 (in 3 folds),0.252 +/- 0.036 (in 3 folds),0.421,0.225,0.420 +/- 0.013 (in 3 folds),0.251 +/- 0.033 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.420,0.225,0.002,Unknown,549,1,550,0.001818,False
ridge_cv,0.753 +/- 0.018 (in 3 folds),0.745 +/- 0.012 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.421 +/- 0.011 (in 3 folds),0.252 +/- 0.036 (in 3 folds),0.421,0.225,0.420 +/- 0.013 (in 3 folds),0.251 +/- 0.033 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.420,0.225,0.002,Unknown,549,1,550,0.001818,False
ridge_cv_decision_thresholds_tuned,0.753 +/- 0.018 (in 3 folds),0.745 +/- 0.012 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.531 +/- 0.054 (in 3 folds),0.407 +/- 0.043 (in 3 folds),0.532,0.393,0.530 +/- 0.056 (in 3 folds),0.405 +/- 0.046 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.531,0.392,0.002,Unknown,549,1,550,0.001818,False
ridge_sklearn_with_lambdamax_decision_thresholds_tuned,0.753 +/- 0.018 (in 3 folds),0.745 +/- 0.012 (in 3 folds),0.769 +/- 0.017 (in 3 folds),0.766 +/- 0.011 (in 3 folds),0.528 +/- 0.049 (in 3 folds),0.402 +/- 0.038 (in 3 folds),0.528,0.386,0.527 +/- 0.050 (in 3 folds),0.400 +/- 0.040 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.527,0.386,0.002,Unknown,549,1,550,0.001818,False
elasticnet0.25_sklearn_with_lambdamax,0.752 +/- 0.005 (in 3 folds),0.743 +/- 0.008 (in 3 folds),0.772 +/- 0.010 (in 3 folds),0.768 +/- 0.009 (in 3 folds),0.415 +/- 0.045 (in 3 folds),0.267 +/- 0.035 (in 3 folds),0.415,0.249,0.415 +/- 0.044 (in 3 folds),0.266 +/- 0.033 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.415,0.248,0.002,Unknown,549,1,550,0.001818,False
elasticnet0.25_sklearn_with_lambdamax_decision_thresholds_tuned,0.752 +/- 0.005 (in 3 folds),0.743 +/- 0.008 (in 3 folds),0.772 +/- 0.010 (in 3 folds),0.768 +/- 0.009 (in 3 folds),0.499 +/- 0.041 (in 3 folds),0.379 +/- 0.060 (in 3 folds),0.499,0.364,0.498 +/- 0.043 (in 3 folds),0.378 +/- 0.061 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.498,0.364,0.002,Unknown,549,1,550,0.001818,False
elasticnet_cv0.25,0.751 +/- 0.005 (in 3 folds),0.742 +/- 0.009 (in 3 folds),0.771 +/- 0.009 (in 3 folds),0.766 +/- 0.008 (in 3 folds),0.419 +/- 0.038 (in 3 folds),0.265 +/- 0.037 (in 3 folds),0.419,0.248,0.418 +/- 0.038 (in 3 folds),0.264 +/- 0.035 (in 3 folds),0.002 +/- 0.003 (in 3 folds),0.418,0.247,0.002,Unknown,549,1,550,0.001818,False
elasticnet_cv0.25_decision_thresholds_tuned,0.751 +/- 0.005 (in 3 folds),0.742 +/- 0.009 (in 3 folds),0.771 +/- 0.009 (in 3 folds),0.766 +/- 0.008 (in 3 folds),0.508 +/- 0.015 (in 3 folds),0.376 +/- 0.026 (in 3 folds),0.508,0.363,0

# Test set performance - summary

In [6]:
for gene_locus in config.gene_loci_used:
    display(Markdown(f"## {gene_locus} test set performance"))
    results_output_dir = ExactMatchesClassifier._get_output_base_dir(
        gene_locus=gene_locus,
        target_obs_column=target_obs_column,
    )
    results_df = pd.read_csv(
        results_output_dir
        / "train_smaller1_model.compare_model_scores.test_set_performance.tsv",
        sep="\t",
        index_col=0,
    )
    show(table(results_df))
    for model_name_chunks in chunks(results_df.index.sort_values(), 4):
        show(
            [
                [
                    results_output_dir
                    / f"train_smaller1_model.test_set_performance.{model_name}.confusion_matrix.png"
                    for model_name in model_name_chunks
                ],
                [
                    results_output_dir
                    / f"train_smaller1_model.test_set_performance.{model_name}.classification_report.txt"
                    for model_name in model_name_chunks
                ],
            ],
            headers=model_name_chunks,
        )

## GeneLocus.BCR test set performance

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
lasso_cv_decision_thresholds_tuned,0.886 +/- 0.043 (in 3 folds),0.870 +/- 0.043 (in 3 folds),0.868 +/- 0.046 (in 3 folds),0.862 +/- 0.047 (in 3 folds),0.683 +/- 0.117 (in 3 folds),0.575 +/- 0.159 (in 3 folds),0.667,0.552,0.399 +/- 0.054 (in 3 folds),0.354 +/- 0.032 (in 3 folds),0.400 +/- 0.150 (in 3 folds),0.399,0.347,0.401,Unknown,369,247,616,0.400974,False
lasso_cv,0.886 +/- 0.043 (in 3 folds),0.870 +/- 0.043 (in 3 folds),0.868 +/- 0.046 (in 3 folds),0.862 +/- 0.047 (in 3 folds),0.705 +/- 0.109 (in 3 folds),0.601 +/- 0.148 (in 3 folds),0.688,0.577,0.413 +/- 0.047 (in 3 folds),0.361 +/- 0.023 (in 3 folds),0.400 +/- 0.150 (in 3 folds),0.412,0.355,0.401,Unknown,369,247,616,0.400974,False
elasticnet_cv_decision_thresholds_tuned,0.886 +/- 0.039 (in 3 folds),0.871 +/- 0.034 (in 3 folds),0.872 +/- 0.040 (in 3 folds),0.865 +/- 0.037 (in 3 folds),0.756 +/- 0.084 (in 3 folds),0.669 +/- 0.119 (in 3 folds),0.741,0.646,0.417 +/- 0.066 (in 3 folds),0.388 +/- 0.011 (in 3 folds),0.436 +/- 0.158 (in 3 folds),0.417,0.381,0.437,Unknown,347,269,616,0.436688,False
elasticnet_cv,0.886 +/- 0.039 (in 3 folds),0.871 +/- 0.034 (in 3 folds),0.872 +/- 0.040 (in 3 folds),0.865 +/- 0.037 (in 3 folds),0.750 +/- 0.112 (in 3 folds),0.660 +/- 0.158 (in 3 folds),0.729,0.632,0.411 +/- 0.050 (in 3 folds),0.380 +/- 0.025 (in 3 folds),0.436 +/- 0.158 (in 3 folds),0.411,0.372,0.437,Unknown,347,269,616,0.436688,False
elasticnet_cv0.25,0.884 +/- 0.037 (in 3 folds),0.870 +/- 0.033 (in 3 folds),0.871 +/- 0.040 (in 3 folds),0.865 +/- 0.038 (in 3 folds),0.747 +/- 0.109 (in 3 folds),0.656 +/- 0.154 (in 3 folds),0.726,0.628,0.409 +/- 0.050 (in 3 folds),0.378 +/- 0.022 (in 3 folds),0.436 +/- 0.158 (in 3 folds),0.409,0.370,0.437,Unknown,347,269,616,0.436688,False
elasticnet_cv0.25_decision_thresholds_tuned,0.884 +/- 0.037 (in 3 folds),0.870 +/- 0.033 (in 3 folds),0.871 +/- 0.040 (in 3 folds),0.865 +/- 0.038 (in 3 folds),0.749 +/- 0.077 (in 3 folds),0.662 +/- 0.113 (in 3 folds),0.735,0.640,0.414 +/- 0.070 (in 3 folds),0.386 +/- 0.020 (in 3 folds),0.436 +/- 0.158 (in 3 folds),0.414,0.379,0.437,Unknown,347,269,616,0.436688,False
logisticregression_unregularized,0.880 +/- 0.048 (in 3 folds),0.866 +/- 0.048 (in 3 folds),0.862 +/- 0.054 (in 3 folds),0.857 +/- 0.055 (in 3 folds),0.693 +/- 0.107 (in 3 folds),0.590 +/- 0.145 (in 3 folds),0.676,0.567,0.424 +/- 0.055 (in 3 folds),0.368 +/- 0.012 (in 3 folds),0.371 +/- 0.162 (in 3 folds),0.424,0.360,0.373,Unknown,386,230,616,0.373377,False
logisticregression_unregularized_decision_thresholds_tuned,0.880 +/- 0.048 (in 3 folds),0.866 +/- 0.048 (in 3 folds),0.862 +/- 0.054 (in 3 folds),0.857 +/- 0.055 (in 3 folds),0.682 +/- 0.100 (in 3 folds),0.596 +/- 0.107 (in 3 folds),0.666,0.571,0.418 +/- 0.057 (in 3 folds),0.374 +/- 0.014 (in 3 folds),0.371 +/- 0.162 (in 3 folds),0.417,0.365,0.373,Unknown,386,230,616,0.373377,False
lasso_sklearn_with_lambdamax_decision_thresholds_tuned,0.878 +/- 0.046 (in 3 folds),0.863 +/- 0.044 (in 3 folds),0.863 +/- 0.050 (in 3 folds),0.857 +/- 0.049 (in 3 folds),0.706 +/- 0.103 (in 3 folds),0.604 +/- 0.144 (in 3 folds),0.689,0.582,0.433 +/- 0.062 (in 3 folds),0.382 +/- 0.031 (in 3 folds),0.371 +/- 0.162 (in 3 folds),0.432,0.375,0.373,Unknown,386,230,616,0.373377,False
lasso_sklearn_with_lambdamax,0.878 +/- 0.046 (in 3 folds),0.863 +/- 0.044 (in 3 folds),0.863 +/- 0.050 (in 3 folds),0.857 +/- 0.049 (in 3 folds),0.708 +/- 0.097 (in 3 folds),0.606 +/- 0.134 (in 3 folds),0.692,0.584,0.4

elasticnet0.25_sklearn_with_lambdamax,elasticnet0.25_sklearn_with_lambdamax_decision_thresholds_tuned,elasticnet0.75_sklearn_with_lambdamax,elasticnet0.75_sklearn_with_lambdamax_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.872 +/- 0.038 (in 3 folds) ROC-AUC (macro OvO): 0.858 +/- 0.038 (in 3 folds) au-PRC (weighted OvO): 0.863 +/- 0.053 (in 3 folds) au-PRC (macro OvO): 0.857 +/- 0.053 (in 3 folds) Accuracy: 0.698 +/- 0.069 (in 3 folds) MCC: 0.591 +/- 0.097 (in 3 folds) Global scores without abstention: Accuracy: 0.687 MCC: 0.575 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.431 +/- 0.077 (in 3 folds) MCC: 0.374 +/- 0.020 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.430 MCC: 0.367 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.89 0.25 0.40 63 HIV 0.81 0.81 0.81 98 Healthy/Background 0.77 0.54 0.63 224 Influenza 0.57 0.11 0.18 37 Lupus 0.56 0.30 0.39 98 T1D 0.30 0.17 0.21 96 Unknown 0.00 0.00 0.00 0 accuracy 0.43 616 macro avg 0.56 0.31 0.37 616 weighted avg 0.67 0.43 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.872 +/- 0.038 (in 3 folds) ROC-AUC (macro OvO): 0.858 +/- 0.038 (in 3 folds) au-PRC (weighted OvO): 0.863 +/- 0.053 (in 3 folds) au-PRC (macro OvO): 0.857 +/- 0.053 (in 3 folds) Accuracy: 0.686 +/- 0.112 (in 3 folds) MCC: 0.596 +/- 0.117 (in 3 folds) Global scores without abstention: Accuracy: 0.671 MCC: 0.570 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.421 +/- 0.070 (in 3 folds) MCC: 0.373 +/- 0.025 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.420 MCC: 0.366 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.84 0.25 0.39 63 HIV 0.81 0.87 0.84 98 Healthy/Background 0.82 0.46 0.59 224 Influenza 0.67 0.11 0.19 37 Lupus 0.41 0.37 0.39 98 T1D 0.36 0.16 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 616 macro avg 0.56 0.32 0.37 616 weighted avg 0.67 0.42 0.49 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.877 +/- 0.044 (in 3 folds) ROC-AUC (macro OvO): 0.863 +/- 0.044 (in 3 folds) au-PRC (weighted OvO): 0.865 +/- 0.055 (in 3 folds) au-PRC (macro OvO): 0.860 +/- 0.054 (in 3 folds) Accuracy: 0.707 +/- 0.078 (in 3 folds) MCC: 0.602 +/- 0.106 (in 3 folds) Global scores without abstention: Accuracy: 0.694 MCC: 0.587 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.436 +/- 0.072 (in 3 folds) MCC: 0.381 +/- 0.014 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.435 MCC: 0.373 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.89 0.25 0.40 63 HIV 0.85 0.81 0.83 98 Healthy/Background 0.78 0.54 0.64 224 Influenza 0.67 0.11 0.19 37 Lupus 0.52 0.33 0.40 98 T1D 0.31 0.17 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.44 616 macro avg 0.57 0.31 0.38 616 weighted avg 0.68 0.44 0.51 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.877 +/- 0.044 (in 3 folds) ROC-AUC (macro OvO): 0.863 +/- 0.044 (in 3 folds) au-PRC (weighted OvO): 0.865 +/- 0.055 (in 3 folds) au-PRC (macro OvO): 0.860 +/- 0.054 (in 3 folds) Accuracy: 0.683 +/- 0.122 (in 3 folds) MCC: 0.593 +/- 0.138 (in 3 folds) Global scores without abstention: Accuracy: 0.666 MCC: 0.566 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.417 +/- 0.063 (in 3 folds) MCC: 0.371 +/- 0.027 (in 3 folds) Unknown/abstention proportion: 0.371 

elasticnet_cv,elasticnet_cv0.25,elasticnet_cv0.25_decision_thresholds_tuned,elasticnet_cv0.25_lambda1se
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.886 +/- 0.039 (in 3 folds) ROC-AUC (macro OvO): 0.871 +/- 0.034 (in 3 folds) au-PRC (weighted OvO): 0.872 +/- 0.040 (in 3 folds) au-PRC (macro OvO): 0.865 +/- 0.037 (in 3 folds) Accuracy: 0.750 +/- 0.112 (in 3 folds) MCC: 0.660 +/- 0.158 (in 3 folds) Global scores without abstention: Accuracy: 0.729 MCC: 0.632 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.411 +/- 0.050 (in 3 folds) MCC: 0.380 +/- 0.025 (in 3 folds) Unknown/abstention proportion: 0.436 +/- 0.158 (in 3 folds) Global scores with abstention: Accuracy: 0.411 MCC: 0.372 Unknown/abstention proportion: 0.437 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.76 0.25 0.38 63 HIV 0.85 0.81 0.83 98 Healthy/Background 0.78 0.48 0.59 224 Influenza 0.50 0.16 0.24 37 Lupus 0.65 0.36 0.46 98 T1D 0.33 0.10 0.16 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 616 macro avg 0.55 0.31 0.38 616 weighted avg 0.68 0.41 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.884 +/- 0.037 (in 3 folds) ROC-AUC (macro OvO): 0.870 +/- 0.033 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.040 (in 3 folds) au-PRC (macro OvO): 0.865 +/- 0.038 (in 3 folds) Accuracy: 0.747 +/- 0.109 (in 3 folds) MCC: 0.656 +/- 0.154 (in 3 folds) Global scores without abstention: Accuracy: 0.726 MCC: 0.628 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.409 +/- 0.050 (in 3 folds) MCC: 0.378 +/- 0.022 (in 3 folds) Unknown/abstention proportion: 0.436 +/- 0.158 (in 3 folds) Global scores with abstention: Accuracy: 0.409 MCC: 0.370 Unknown/abstention proportion: 0.437 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.76 0.25 0.38 63 HIV 0.87 0.81 0.84 98 Healthy/Background 0.77 0.47 0.59 224 Influenza 0.46 0.16 0.24 37 Lupus 0.64 0.36 0.46 98 T1D 0.33 0.10 0.16 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 616 macro avg 0.55 0.31 0.38 616 weighted avg 0.68 0.41 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.884 +/- 0.037 (in 3 folds) ROC-AUC (macro OvO): 0.870 +/- 0.033 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.040 (in 3 folds) au-PRC (macro OvO): 0.865 +/- 0.038 (in 3 folds) Accuracy: 0.749 +/- 0.077 (in 3 folds) MCC: 0.662 +/- 0.113 (in 3 folds) Global scores without abstention: Accuracy: 0.735 MCC: 0.640 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.414 +/- 0.070 (in 3 folds) MCC: 0.386 +/- 0.020 (in 3 folds) Unknown/abstention proportion: 0.436 +/- 0.158 (in 3 folds) Global scores with abstention: Accuracy: 0.414 MCC: 0.379 Unknown/abstention proportion: 0.437 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.80 0.25 0.39 63 HIV 0.79 0.83 0.81 98 Healthy/Background 0.82 0.48 0.61 224 Influenza 0.46 0.16 0.24 37 Lupus 0.60 0.34 0.43 98 T1D 0.44 0.11 0.18 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 616 macro avg 0.56 0.31 0.38 616 weighted avg 0.70 0.41 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.862 +/- 0.029 (in 3 folds) ROC-AUC (macro OvO): 0.850 +/- 0.034 (in 3 folds) au-PRC (weighted OvO): 0.850 +/- 0.036 (in 3 folds) au-PRC (macro OvO): 0.847 +/- 0.043 (in 3 folds) Accuracy: 0.681 +/- 0.052 (in 3 folds) MCC: 0.563 +/- 0.073 (in 3 folds) Global scores without abstention: Accuracy: 0.674 MCC: 0.551 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.423 +/- 0.086 (in 3 folds) MCC: 0.353 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.422 MCC: 0.348 Unknown/abstention 

elasticnet_cv0.25_lambda1se_decision_thresholds_tuned,elasticnet_cv0.75,elasticnet_cv0.75_decision_thresholds_tuned,elasticnet_cv0.75_lambda1se
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.862 +/- 0.029 (in 3 folds) ROC-AUC (macro OvO): 0.850 +/- 0.034 (in 3 folds) au-PRC (weighted OvO): 0.850 +/- 0.036 (in 3 folds) au-PRC (macro OvO): 0.847 +/- 0.043 (in 3 folds) Accuracy: 0.685 +/- 0.069 (in 3 folds) MCC: 0.583 +/- 0.092 (in 3 folds) Global scores without abstention: Accuracy: 0.674 MCC: 0.567 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.423 +/- 0.074 (in 3 folds) MCC: 0.369 +/- 0.022 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.422 MCC: 0.363 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.64 0.29 0.40 63 HIV 0.83 0.80 0.81 98 Healthy/Background 0.81 0.50 0.62 224 Influenza 0.57 0.11 0.18 37 Lupus 0.44 0.34 0.38 98 T1D 0.35 0.16 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 616 macro avg 0.52 0.31 0.37 616 weighted avg 0.65 0.42 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.877 +/- 0.044 (in 3 folds) ROC-AUC (macro OvO): 0.862 +/- 0.044 (in 3 folds) au-PRC (weighted OvO): 0.861 +/- 0.048 (in 3 folds) au-PRC (macro OvO): 0.856 +/- 0.049 (in 3 folds) Accuracy: 0.709 +/- 0.107 (in 3 folds) MCC: 0.607 +/- 0.146 (in 3 folds) Global scores without abstention: Accuracy: 0.692 MCC: 0.585 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.434 +/- 0.059 (in 3 folds) MCC: 0.379 +/- 0.020 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.433 MCC: 0.371 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.84 0.25 0.39 63 HIV 0.88 0.79 0.83 98 Healthy/Background 0.77 0.54 0.63 224 Influenza 0.67 0.11 0.19 37 Lupus 0.54 0.36 0.43 98 T1D 0.28 0.16 0.20 96 Unknown 0.00 0.00 0.00 0 accuracy 0.43 616 macro avg 0.57 0.31 0.38 616 weighted avg 0.68 0.43 0.51 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.877 +/- 0.044 (in 3 folds) ROC-AUC (macro OvO): 0.862 +/- 0.044 (in 3 folds) au-PRC (weighted OvO): 0.861 +/- 0.048 (in 3 folds) au-PRC (macro OvO): 0.856 +/- 0.049 (in 3 folds) Accuracy: 0.700 +/- 0.096 (in 3 folds) MCC: 0.596 +/- 0.136 (in 3 folds) Global scores without abstention: Accuracy: 0.684 MCC: 0.574 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.429 +/- 0.063 (in 3 folds) MCC: 0.376 +/- 0.017 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accuracy: 0.429 MCC: 0.368 Unknown/abstention proportion: 0.373 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.84 0.25 0.39 63 HIV 0.84 0.84 0.84 98 Healthy/Background 0.79 0.53 0.63 224 Influenza 0.67 0.11 0.19 37 Lupus 0.43 0.38 0.40 98 T1D 0.27 0.07 0.11 96 Unknown 0.00 0.00 0.00 0 accuracy 0.43 616 macro avg 0.55 0.31 0.37 616 weighted avg 0.65 0.43 0.50 616,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.860 +/- 0.028 (in 3 folds) ROC-AUC (macro OvO): 0.849 +/- 0.031 (in 3 folds) au-PRC (weighted OvO): 0.849 +/- 0.035 (in 3 folds) au-PRC (macro OvO): 0.848 +/- 0.040 (in 3 folds) Accuracy: 0.675 +/- 0.047 (in 3 folds) MCC: 0.556 +/- 0.064 (in 3 folds) Global scores without abstention: Accuracy: 0.668 MCC: 0.544 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.420 +/- 0.088 (in 3 folds) MCC: 0.349 +/- 0.047 (in 3 folds) Unknown/abstention proportion: 0.371 +/- 0.162 (in 3 folds) Global scores with abstention: Accura

...

lasso_cv_lambda1se,lasso_cv_lambda1se_decision_thresholds_tuned,lasso_sklearn_with_lambdamax,lasso_sklearn_with_lambdamax_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.740 +/- 0.008 (in 3 folds) ROC-AUC (macro OvO): 0.728 +/- 0.016 (in 3 folds) au-PRC (weighted OvO): 0.762 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.756 +/- 0.010 (in 3 folds) Accuracy: 0.383 +/- 0.082 (in 3 folds) MCC: 0.250 +/- 0.049 (in 3 folds) Global scores without abstention: Accuracy: 0.383 MCC: 0.225 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.382 +/- 0.082 (in 3 folds) MCC: 0.248 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.382 MCC: 0.224 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.15 0.59 0.24 58 HIV 0.77 0.17 0.28 98 Healthy/Background 0.51 0.66 0.58 197 Influenza 1.00 0.16 0.28 37 Lupus 0.92 0.19 0.31 64 T1D 0.48 0.11 0.18 96 Unknown 0.00 0.00 0.00 0 accuracy 0.38 550 macro avg 0.55 0.27 0.27 550 weighted avg 0.59 0.38 0.37 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.740 +/- 0.008 (in 3 folds) ROC-AUC (macro OvO): 0.728 +/- 0.016 (in 3 folds) au-PRC (weighted OvO): 0.762 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.756 +/- 0.010 (in 3 folds) Accuracy: 0.484 +/- 0.030 (in 3 folds) MCC: 0.331 +/- 0.031 (in 3 folds) Global scores without abstention: Accuracy: 0.485 MCC: 0.318 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.483 +/- 0.030 (in 3 folds) MCC: 0.330 +/- 0.031 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.484 MCC: 0.318 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.22 0.29 0.25 58 HIV 0.66 0.60 0.63 98 Healthy/Background 0.53 0.75 0.62 197 Influenza 1.00 0.22 0.36 37 Lupus 0.45 0.30 0.36 64 T1D 0.31 0.17 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.48 550 macro avg 0.45 0.33 0.35 550 weighted avg 0.50 0.48 0.46 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.746 +/- 0.006 (in 3 folds) ROC-AUC (macro OvO): 0.736 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.765 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.760 +/- 0.010 (in 3 folds) Accuracy: 0.408 +/- 0.062 (in 3 folds) MCC: 0.264 +/- 0.042 (in 3 folds) Global scores without abstention: Accuracy: 0.408 MCC: 0.243 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.407 +/- 0.062 (in 3 folds) MCC: 0.262 +/- 0.040 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.407 MCC: 0.242 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.15 0.52 0.24 58 HIV 0.79 0.19 0.31 98 Healthy/Background 0.51 0.71 0.59 197 Influenza 1.00 0.16 0.28 37 Lupus 0.93 0.20 0.33 64 T1D 0.47 0.18 0.26 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 550 macro avg 0.55 0.28 0.29 550 weighted avg 0.60 0.41 0.39 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.746 +/- 0.006 (in 3 folds) ROC-AUC (macro OvO): 0.736 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.765 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.760 +/- 0.010 (in 3 folds) Accuracy: 0.510 +/- 0.019 (in 3 folds) MCC: 0.366 +/- 0.030 (in 3 folds) Global scores without abstention: Accuracy: 0.510 MCC: 0.358 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.509 +/- 0.020 (in 3 folds) MCC: 0.365 +/- 0.031 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Ac

linearsvm_ovr,linearsvm_ovr_decision_thresholds_tuned,logisticregression_unregularized,logisticregression_unregularized_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.742 +/- 0.024 (in 3 folds) ROC-AUC (macro OvO): 0.736 +/- 0.024 (in 3 folds) au-PRC (weighted OvO): 0.755 +/- 0.030 (in 3 folds) au-PRC (macro OvO): 0.754 +/- 0.030 (in 3 folds) Accuracy: 0.460 +/- 0.008 (in 3 folds) MCC: 0.289 +/- 0.022 (in 3 folds) Global scores without abstention: Accuracy: 0.460 MCC: 0.283 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.458 +/- 0.006 (in 3 folds) MCC: 0.287 +/- 0.020 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.458 MCC: 0.282 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.21 0.34 0.26 58 HIV 0.74 0.38 0.50 98 Healthy/Background 0.51 0.78 0.61 197 Influenza 0.90 0.24 0.38 37 Lupus 0.52 0.23 0.32 64 T1D 0.29 0.18 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.46 550 macro avg 0.45 0.31 0.33 550 weighted avg 0.51 0.46 0.44 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.742 +/- 0.024 (in 3 folds) ROC-AUC (macro OvO): 0.736 +/- 0.024 (in 3 folds) au-PRC (weighted OvO): 0.755 +/- 0.030 (in 3 folds) au-PRC (macro OvO): 0.754 +/- 0.030 (in 3 folds) Accuracy: 0.491 +/- 0.037 (in 3 folds) MCC: 0.355 +/- 0.071 (in 3 folds) Global scores without abstention: Accuracy: 0.491 MCC: 0.331 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.489 +/- 0.038 (in 3 folds) MCC: 0.354 +/- 0.071 (in 3 folds) Unknown/abstention proportion: 0.004 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.489 MCC: 0.330 Unknown/abstention proportion: 0.004 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.38 0.19 0.25 58 HIV 0.56 0.63 0.60 98 Healthy/Background 0.54 0.70 0.60 197 Influenza 0.70 0.38 0.49 37 Lupus 0.36 0.47 0.41 64 T1D 0.31 0.16 0.21 96 Unknown 0.00 0.00 0.00 0 accuracy 0.49 550 macro avg 0.41 0.36 0.37 550 weighted avg 0.47 0.49 0.47 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.746 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.735 +/- 0.008 (in 3 folds) au-PRC (weighted OvO): 0.768 +/- 0.010 (in 3 folds) au-PRC (macro OvO): 0.763 +/- 0.011 (in 3 folds) Accuracy: 0.406 +/- 0.060 (in 3 folds) MCC: 0.259 +/- 0.044 (in 3 folds) Global scores without abstention: Accuracy: 0.406 MCC: 0.242 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.406 +/- 0.059 (in 3 folds) MCC: 0.258 +/- 0.041 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.405 MCC: 0.241 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.15 0.52 0.24 58 HIV 0.80 0.16 0.27 98 Healthy/Background 0.51 0.70 0.59 197 Influenza 1.00 0.16 0.28 37 Lupus 0.92 0.19 0.31 64 T1D 0.45 0.22 0.29 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 550 macro avg 0.55 0.28 0.28 550 weighted avg 0.60 0.41 0.39 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.746 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.735 +/- 0.008 (in 3 folds) au-PRC (weighted OvO): 0.768 +/- 0.010 (in 3 folds) au-PRC (macro OvO): 0.763 +/- 0.011 (in 3 folds) Accuracy: 0.500 +/- 0.035 (in 3 folds) MCC: 0.381 +/- 0.052 (in 3 folds) Global scores without abstention: Accuracy: 0.501 MCC: 0.369 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.500 +/- 0.036 (in 3 folds) MCC: 0.380 +/- 0.053 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accu

rf_multiclass,rf_multiclass_decision_thresholds_tuned,ridge_cv,ridge_cv_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.701 +/- 0.056 (in 3 folds) ROC-AUC (macro OvO): 0.694 +/- 0.068 (in 3 folds) au-PRC (weighted OvO): 0.713 +/- 0.051 (in 3 folds) au-PRC (macro OvO): 0.712 +/- 0.060 (in 3 folds) Accuracy: 0.417 +/- 0.055 (in 3 folds) MCC: 0.239 +/- 0.075 (in 3 folds) Global scores without abstention: Accuracy: 0.417 MCC: 0.236 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.415 +/- 0.055 (in 3 folds) MCC: 0.237 +/- 0.074 (in 3 folds) Unknown/abstention proportion: 0.005 +/- 0.005 (in 3 folds) Global scores with abstention: Accuracy: 0.415 MCC: 0.234 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.17 0.36 0.23 58 HIV 0.68 0.23 0.35 98 Healthy/Background 0.52 0.72 0.60 197 Influenza 0.79 0.30 0.43 37 Lupus 0.77 0.16 0.26 64 T1D 0.24 0.23 0.23 96 Unknown 0.00 0.00 0.00 0 accuracy 0.41 550 macro avg 0.45 0.29 0.30 550 weighted avg 0.51 0.41 0.40 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.701 +/- 0.056 (in 3 folds) ROC-AUC (macro OvO): 0.694 +/- 0.068 (in 3 folds) au-PRC (weighted OvO): 0.713 +/- 0.051 (in 3 folds) au-PRC (macro OvO): 0.712 +/- 0.060 (in 3 folds) Accuracy: 0.432 +/- 0.041 (in 3 folds) MCC: 0.267 +/- 0.069 (in 3 folds) Global scores without abstention: Accuracy: 0.431 MCC: 0.251 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.429 +/- 0.038 (in 3 folds) MCC: 0.265 +/- 0.067 (in 3 folds) Unknown/abstention proportion: 0.005 +/- 0.005 (in 3 folds) Global scores with abstention: Accuracy: 0.429 MCC: 0.250 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.18 0.12 0.15 58 HIV 0.66 0.26 0.37 98 Healthy/Background 0.54 0.70 0.61 197 Influenza 0.81 0.35 0.49 37 Lupus 0.62 0.20 0.31 64 T1D 0.23 0.43 0.30 96 Unknown 0.00 0.00 0.00 0 accuracy 0.43 550 macro avg 0.43 0.29 0.32 550 weighted avg 0.50 0.43 0.42 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.421 +/- 0.011 (in 3 folds) MCC: 0.252 +/- 0.036 (in 3 folds) Global scores without abstention: Accuracy: 0.421 MCC: 0.225 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.420 +/- 0.013 (in 3 folds) MCC: 0.251 +/- 0.033 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.420 MCC: 0.225 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.21 0.43 0.28 58 HIV 0.89 0.16 0.28 98 Healthy/Background 0.44 0.83 0.58 197 Influenza 1.00 0.16 0.28 37 Lupus 0.92 0.17 0.29 64 T1D 0.43 0.09 0.15 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 550 macro avg 0.55 0.26 0.27 550 weighted avg 0.59 0.42 0.36 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.531 +/- 0.054 (in 3 folds) MCC: 0.407 +/- 0.043 (in 3 folds) Global scores without abstention: Accuracy: 0.532 MCC: 0.393 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.530 +/- 0.056 (in 3 folds) MCC: 0.405 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.531 MCC: 0.392 Unknown/abstention propor

ridge_cv_lambda1se,ridge_cv_lambda1se_decision_thresholds_tuned,ridge_sklearn_with_lambdamax,ridge_sklearn_with_lambdamax_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.013 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.421 +/- 0.011 (in 3 folds) MCC: 0.252 +/- 0.036 (in 3 folds) Global scores without abstention: Accuracy: 0.421 MCC: 0.224 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.420 +/- 0.013 (in 3 folds) MCC: 0.251 +/- 0.033 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.420 MCC: 0.223 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.22 0.43 0.29 58 HIV 0.89 0.16 0.28 98 Healthy/Background 0.44 0.83 0.57 197 Influenza 1.00 0.16 0.28 37 Lupus 0.92 0.17 0.29 64 T1D 0.43 0.09 0.15 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 550 macro avg 0.56 0.26 0.27 550 weighted avg 0.59 0.42 0.36 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.013 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.533 +/- 0.054 (in 3 folds) MCC: 0.412 +/- 0.046 (in 3 folds) Global scores without abstention: Accuracy: 0.534 MCC: 0.401 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.532 +/- 0.055 (in 3 folds) MCC: 0.411 +/- 0.048 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.533 MCC: 0.400 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.28 0.41 0.33 58 HIV 0.60 0.69 0.64 98 Healthy/Background 0.61 0.73 0.66 197 Influenza 0.92 0.32 0.48 37 Lupus 0.41 0.52 0.46 64 T1D 0.59 0.14 0.22 96 Unknown 0.00 0.00 0.00 0 accuracy 0.53 550 macro avg 0.49 0.40 0.40 550 weighted avg 0.57 0.53 0.51 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.421 +/- 0.011 (in 3 folds) MCC: 0.252 +/- 0.036 (in 3 folds) Global scores without abstention: Accuracy: 0.421 MCC: 0.225 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.420 +/- 0.013 (in 3 folds) MCC: 0.251 +/- 0.033 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Accuracy: 0.420 MCC: 0.225 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.21 0.43 0.28 58 HIV 0.89 0.16 0.28 98 Healthy/Background 0.44 0.83 0.58 197 Influenza 1.00 0.16 0.28 37 Lupus 0.92 0.17 0.29 64 T1D 0.43 0.09 0.15 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 550 macro avg 0.55 0.26 0.27 550 weighted avg 0.59 0.42 0.36 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.753 +/- 0.018 (in 3 folds) ROC-AUC (macro OvO): 0.745 +/- 0.012 (in 3 folds) au-PRC (weighted OvO): 0.769 +/- 0.017 (in 3 folds) au-PRC (macro OvO): 0.766 +/- 0.011 (in 3 folds) Accuracy: 0.528 +/- 0.049 (in 3 folds) MCC: 0.402 +/- 0.038 (in 3 folds) Global scores without abstention: Accuracy: 0.528 MCC: 0.386 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.527 +/- 0.050 (in 3 folds) MCC: 0.400 +/- 0.040 (in 3 folds) Unknown/abstention proportion: 0.002 +/- 0.003 (in 3 folds) Global scores with abstention: Ac

xgboost,xgboost_decision_thresholds_tuned
,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.695 +/- 0.047 (in 3 folds) ROC-AUC (macro OvO): 0.689 +/- 0.048 (in 3 folds) au-PRC (weighted OvO): 0.707 +/- 0.046 (in 3 folds) au-PRC (macro OvO): 0.707 +/- 0.049 (in 3 folds) Accuracy: 0.424 +/- 0.037 (in 3 folds) MCC: 0.248 +/- 0.048 (in 3 folds) Global scores without abstention: Accuracy: 0.424 MCC: 0.235 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.422 +/- 0.038 (in 3 folds) MCC: 0.246 +/- 0.048 (in 3 folds) Unknown/abstention proportion: 0.005 +/- 0.005 (in 3 folds) Global scores with abstention: Accuracy: 0.422 MCC: 0.234 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.23 0.24 0.23 58 HIV 0.49 0.24 0.33 98 Healthy/Background 0.51 0.70 0.59 197 Influenza 0.79 0.30 0.43 37 Lupus 0.68 0.20 0.31 64 T1D 0.25 0.34 0.29 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 550 macro avg 0.42 0.29 0.31 550 weighted avg 0.47 0.42 0.41 550,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.695 +/- 0.047 (in 3 folds) ROC-AUC (macro OvO): 0.689 +/- 0.048 (in 3 folds) au-PRC (weighted OvO): 0.707 +/- 0.046 (in 3 folds) au-PRC (macro OvO): 0.707 +/- 0.049 (in 3 folds) Accuracy: 0.421 +/- 0.036 (in 3 folds) MCC: 0.255 +/- 0.066 (in 3 folds) Global scores without abstention: Accuracy: 0.420 MCC: 0.236 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.418 +/- 0.038 (in 3 folds) MCC: 0.254 +/- 0.066 (in 3 folds) Unknown/abstention proportion: 0.005 +/- 0.005 (in 3 folds) Global scores with abstention: Accuracy: 0.418 MCC: 0.235 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.28 0.28 0.28 58 HIV 0.45 0.24 0.32 98 Healthy/Background 0.53 0.68 0.59 197 Influenza 0.38 0.30 0.33 37 Lupus 0.68 0.20 0.31 64 T1D 0.24 0.33 0.28 96 Unknown 0.00 0.00 0.00 0 accuracy 0.42 550 macro avg 0.37 0.29 0.30 550 weighted avg 0.45 0.42 0.41 550


# Compare to CDR3 clustering

In [7]:
for gene_locus in config.gene_loci_used:
    results_output_prefix_alternative = (
        ConvergentClusterClassifier._get_output_base_dir(
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
        )
        / "train_smaller1_model"
    )

    display(
        Markdown(
            f"## Compare to CDR3 clustering: {gene_locus}, {target_obs_column}, eval on test set, with and without tuning on validation set"
        )
    )
    try:
        ## All results in a table
        all_results = pd.read_csv(
            f"{results_output_prefix_alternative}.compare_model_scores.test_set_performance.tsv",
            sep="\t",
            index_col=0,
        )
        show(table(all_results), headers=["All results, sorted"])

        models_of_interest = all_results.index

        ## Confusion matrices
        for model_names in chunks(models_of_interest, 4):
            show(
                [
                    [
                        f"{results_output_prefix_alternative}.test_set_performance.{model_name}.classification_report.txt"
                        for model_name in model_names
                    ],
                    [
                        f"{results_output_prefix_alternative}.test_set_performance.{model_name}.confusion_matrix.png"
                        for model_name in model_names
                    ],
                ],
                max_width=500,
                headers=model_names,
            )
    except FileNotFoundError as err:
        logger.warning(f"Not run: {err}")

    display(Markdown("---"))

## Compare to CDR3 clustering: GeneLocus.BCR, TargetObsColumnEnum.disease, eval on test set, with and without tuning on validation set

{"message": "Not run: [Errno 2] No such file or directory: '/home/maxim/code/immune-repertoire-classification/out/in_house_peak_disease_timepoints/convergent_clusters/BCR/disease/train_smaller1_model.compare_model_scores.test_set_performance.tsv'", "time": "2023-11-04T06:47:45.017009"}


---

## Compare to CDR3 clustering: GeneLocus.TCR, TargetObsColumnEnum.disease, eval on test set, with and without tuning on validation set

{"message": "Not run: [Errno 2] No such file or directory: '/home/maxim/code/immune-repertoire-classification/out/in_house_peak_disease_timepoints/convergent_clusters/TCR/disease/train_smaller1_model.compare_model_scores.test_set_performance.tsv'", "time": "2023-11-04T06:47:45.056317"}


---

# Which p values were chosen (varies by locus, model, and fold)? How many disease-associated sequences found?

In [8]:
for gene_locus in config.gene_loci_used:
    for fold_id in config.cross_validation_fold_ids:
        for model_name in model_names_to_check_later:
            try:
                clf = ExactMatchesClassifier(
                    fold_id=fold_id,
                    model_name=model_name,
                    fold_label_train="train_smaller1",
                    target_obs_column=target_obs_column,
                    gene_locus=gene_locus,
                )
                p_value = clf.p_value_threshold
                seqs = clf.sequences_with_fisher_result
                print(
                    f"{gene_locus}, fold {fold_id}, {model_name}: p = {p_value}. Number of {target_obs_column.name} associated sequences: {(seqs <= p_value).sum().to_dict()}"
                )
            except FileNotFoundError as err:
                logger.warning(f"Not run: {err}")
        print()

GeneLocus.BCR, fold 0, ridge_sklearn_with_lambdamax: p = 0.01. Number of disease associated sequences: {'Covid19': 11, 'HIV': 81, 'Healthy/Background': 16, 'Influenza': 96, 'Lupus': 15, 'T1D': 2}


GeneLocus.BCR, fold 0, ridge_cv: p = 0.01. Number of disease associated sequences: {'Covid19': 11, 'HIV': 81, 'Healthy/Background': 16, 'Influenza': 96, 'Lupus': 15, 'T1D': 2}


GeneLocus.BCR, fold 0, ridge_cv_lambda1se: p = 0.01. Number of disease associated sequences: {'Covid19': 11, 'HIV': 81, 'Healthy/Background': 16, 'Influenza': 96, 'Lupus': 15, 'T1D': 2}


GeneLocus.BCR, fold 0, elasticnet0.25_sklearn_with_lambdamax: p = 0.01. Number of disease associated sequences: {'Covid19': 11, 'HIV': 81, 'Healthy/Background': 16, 'Influenza': 96, 'Lupus': 15, 'T1D': 2}


GeneLocus.BCR, fold 0, elasticnet_sklearn_with_lambdamax: p = 0.01. Number of disease associated sequences: {'Covid19': 11, 'HIV': 81, 'Healthy/Background': 16, 'Influenza': 96, 'Lupus': 15, 'T1D': 2}


...

GeneLocus.TCR, fold 2, lasso_cv: p = 0.001. Number of disease associated sequences: {'Covid19': 0, 'HIV': 112, 'Healthy/Background': 371, 'Influenza': 84, 'Lupus': 214, 'T1D': 6}


GeneLocus.TCR, fold 2, elasticnet_cv_each_disease_versus_healthy: p = 0.001. Number of disease associated sequences: {'Covid19': 0, 'HIV': 112, 'Healthy/Background': 371, 'Influenza': 84, 'Lupus': 214, 'T1D': 6}


GeneLocus.TCR, fold 2, xgboost: p = 0.001. Number of disease associated sequences: {'Covid19': 0, 'HIV': 112, 'Healthy/Background': 371, 'Influenza': 84, 'Lupus': 214, 'T1D': 6}


GeneLocus.TCR, fold 2, rf_multiclass: p = 0.001. Number of disease associated sequences: {'Covid19': 0, 'HIV': 112, 'Healthy/Background': 371, 'Influenza': 84, 'Lupus': 214, 'T1D': 6}


GeneLocus.TCR, fold 2, elasticnet_each_disease_versus_healthy_sklearn_with_lambdamax: p = 0.001. Number of disease associated sequences: {'Covid19': 0, 'HIV': 112, 'Healthy/Background': 371, 'Influenza': 84, 'Lupus': 214, 'T1D': 6}

